**1. transforms 是图片操作工具集，具体的 transforms 方法（例如 train_transform）定义在训练文件中：**

```
train.py

train_transform = transforms.Compose([
    transforms.Resize((32, 32)),  # 重设图片大小
    transforms.RandomCrop(32, padding=4),  # 对图片进行裁剪，数据增强
    transforms.ToTensor(),  # 将图片数据转换为 tensor 格式，同时将像素值从 0~255 归一化为 0~1
    transforms.Normalize(norm_mean, norm_std),  # 根据给定的均值和标准差将数据标准化
])
···

train_data = myDataset(data_dir=train_dir, transform=train_transform)
```

<br/>

***

**2. 在 `__getitem__` 中调用 transform，实现对图片数据的预处理：**

```
class myDataset(Dataset):
    def __init__(self, data_dir, transform=None):

        ···

    def __getitem__(self, index):

        ···

        if self.transform is not None:
            img = self.transform(img)   # 调用 实现对图片数据的预处理

        return img, label
```

**transform 的数据逻辑：**

![](./图1.png)

<br/>

***

**3. transforms 的方法**

**3.1 `transforms.Normalize(mean, std, inplace=Flase)`**

功能：逐 channel 地对图像数据进行标准化，inplace 为是否原地操作，默认为 Flase。

例如：`transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225], inplace=Flase)`

数据进行标准化的优点：加快模型的收敛

<br/>

**3.2 `transforms.Resize(size)`**

功能：重设图片大小

size：图片期望输出大小，是一个元组

<br/>

**3.3 `transforms.ToTensor()`**

功能：将图片数据转换为 tensor 格式，同时将像素值从 0~255 归一化为 0~1

<br/>

***

**4. transforms 的数据增强方法**

**数据增强：也称为数据增广，通过对训练集数据进行变换来实现数据增广，扩充更多更丰富的训练集数据，提高模型的泛化能力。**

<br/>

**4.1 `transforms.CenterCrop(size)`**

功能：从图像中心裁剪图片

size：图片需裁剪尺寸

<br/>

**4.2 `transforms.RandomCrop(size, padding=None, pad_if_needed=False, fill=0, padding_mode="constant")`**

功能：将图片**随机裁剪**出尺寸为 size 的图片

size：图片需要裁剪的目标尺寸；

padding：设置填充大小，有三种形式：

    当为 a 时，上下左右均填充 a 个像素
    当为 (a, b) 时，上下填充 b 个像素，左右填充 a 个像素
    当为 (a, b, c, d) 时，左上右下分别填充a, b, c, d个像素

pad_if_need：若图像小于设定的 size，必须设置为 True

padding_mode：填充模式，有 4 种模式；

constant：像素值由 fill 设定；

edge：像素值由图像边缘像素值决定；

reflect：镜像填充，最后一个像素不镜像，例如，**1234** ——> 432**1234**321；

symmetric：镜像填充，最后一个像素镜像，例如，**1234** ——> 4321**1234**4321；

fill：padding_mode 为 constant 时才需要设置，为填充的像素值。

<br/>

**4.3 `transforms.RandomResizedCrop(size, scale=(0.08, 1.0), ratio=(3/4, 4/3), interpolation)`**

功能：**随机大小、长宽比裁剪**图片

size：图片需要裁剪的目标尺寸

scale：随机裁剪面积比例（占原图像的比例），默认区间为 (0.08, 1.0)

ratio：随机长宽比，默认比例区间为 (3/4, 4/3)

interpolation：裁剪之后若图像大小小于设置的 size，则进行插值，有三种插值方法：

    PIL.Image.NEARST
    PIL.Image.BILINEAR
    PIL.Image.BICUBIC

<br/>

**4.4 `transforms.RandomHorizontalFlip(p=0.5)`**

功能：按照一定概率水平对图片进行**水平（左右）翻转**

p：翻转概率，决定选取多少图片进行翻转，p=0.5 表示一半的图片翻转，一半的图片不翻转。

<br/>

**4.5 `transforms.RandomVerticalFlip(p=0.5)`**

功能：按照一定概率水平对图片进行**垂直（上下）翻转**

p：翻转概率，决定选取多少图片进行翻转，p=0.5 表示一半的图片翻转，一半的图片不翻转。

<br/>

**4.6 `transforms.RandomRotation(degrees, expand=False)`**

功能：随机**旋转**图片

degrees：旋转角度

    当值为 a 时，在 (-a, a) 之间随机选择旋转角度；
    当值为 (a, b) 时，在 (a, b) 之间随机选择旋转角度。

expand：图片旋转后，边角处可能超出原图片框范围，设置 expand=True 将会扩大图片框，以保持原图信息。

<br/>

**4.7 `transforms.Pad(padding, fill=0, padding_mode="constant")`**

功能：对**图片边缘进行填充**

参数介绍参考 4.2 `transforms.RandomCrop()`

<br/>

**4.8 `transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=0)`**

功能：调整图片**亮度、对比度、饱和度和色调**

brightness：亮度调节因子，当为 a 时，从 `[max(0, 1-a), 1+a]` 中随机选择值；当为 (a, b) 时，从 `[a, b]` 中随机选择值

contrast：对比度参数，同 brightness

saturation：饱和度参数，同 brightness

hue：色调参数，当为 a 时，从 `[-a, a]` 中随机选择值（注：0 <= a <= 0.5）；当为 (a, b) 时，从 `[a, b]` 中随机选择值（注：0 <= a <= b <= 0.5）

<br/>

**4.9 `transforms.Grayscale(num_output_channels)`**

功能：将（全部）图片转换为**灰度图**，相当于 p=1

num_output_channels：输出通道数，只能为 1 或 3

<br/>

**4.10 `transforms.RandomGrayscale(num_output_channels, p=0.1)`**

功能：依概率将图片转换为灰度图

num_output_channels：输出通道数，只能为 1 或 3

p：图像被转换为灰度图的概率